In [115]:
# this is the module that we created
from my_encoder import Myencoder

In [56]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## For this illustration i am using the titanic dataset

In [123]:
data = pd.read_csv(r'E:\programming\dataset\titanic_kaggle\train.csv')
print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Before moving further, check for the missing values


In [130]:
data.isnull().sum()
data['Age'] = data['Age'].fillna(data['Age'].mean())
data.drop('Cabin',axis = 1,inplace = True)
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

### Get title is used to get titles from names

In [131]:
def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'


In [132]:
data['Name'] = data['Name'].apply(get_title)

In [133]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,Mrs,female,35.0,1,0,113803,53.1000,S
4,5,0,3,Mr,male,35.0,0,0,373450,8.0500,S


## We will see the how encoding is done is Onehotencoding,and dummy variables, and see their advantages and disadvantages, then we will use our module for covering for the disadvantages

# We will use ['Name,'Sex','Embarked'] as the features for encoding

### But first, its always better to split the data into train and test

In [134]:
X_train,X_test,y_train,y_test = train_test_split(data.drop('Survived',axis = 1),data['Survived'],test_size = 0.3,random_state = 1)

In [201]:
# we are going to remove all the observations in test data which have value of Embarked as 'Q'
# we will see later why we did this
X_test = X_test[X_test['Embarked']!='Q']

In [202]:
print('Shape of X_train ',X_train.shape)
print('Shape of X_Test ',X_test.shape)

Shape of X_train  (623, 10)
Shape of X_Test  (243, 10)


In [203]:
features_to_encode = ['Name','Sex','Embarked']

# One Hot Encoder

In [204]:
# in this case, we will first need to separate the dataset such that the varibles which are to be encoded are together.
# since we have the list, its easy (features_to_encode)
# next step it to apply the encoding to the part which is to be encoded
# then combine the two

In [205]:
X_train_to_be_encoded = X_train[features_to_encode]
X_train_left = X_train.drop(features_to_encode,axis = 1)

In [206]:
# now apply encoding to the part of the data that is to be encoded

ohe = OneHotEncoder(drop='first',sparse = False) # drops the first col, sparse = False, returns numpy array, if True,it returns
                                                #sparse array
ohe.fit(X_train_to_be_encoded)
X_train_encoded_ohe = ohe.transform(X_train_to_be_encoded)

In [207]:
# now join them again
# but wait, here comes the disadvantage associated with onehotencoders
type(X_train_encoded_ohe) # they are arrays, not dataframes, hence we cannot add them back

numpy.ndarray

### Disadvantage - Returns arrays

In [208]:
# but we can convert arrays to dataframs, right?
# lets convert it to dataframe and add them together
X_train_array_to_df = pd.DataFrame(X_train_encoded_ohe)

#before concatenating, reset the indices
X_train_array_to_df.reset_index(drop = True,inplace = True)
X_train_left.reset_index(drop = True,inplace = True)

X_train_ohe = pd.concat([X_train_left,X_train_array_to_df],axis = 1)

In [209]:
print(X_train_ohe.shape)
X_train_ohe.head()

(623, 14)


,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,0,1,2,3,4,5,6
0,115,3,17.000000,0,0,2627,14.4583,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,875,2,28.000000,1,0,P/PP 3381,24.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,77,3,29.699118,0,0,349208,7.8958,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,877,3,20.000000,0,0,7534,9.8458,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,675,2,29.699118,0,0,239856,0.0000,0.0,1.0,0.0,0.0,1.0,0.0,1.0


### Finally, after writing all these line of codes i have fnally got what i wanted, but wait again,
### where are the names of the encoded columns, how can i know what is describing what?

## Disadvantage - Columns name information is lost

# DUMMIES

In [210]:
# Now lets use dummies to see what it has to offer

In [211]:
# the approach is same, we first divide it into two parts, encode the part which is to be encoded, and then combine 
# to the part which was left behind in the first case

In [212]:
X_train_to_be_encoded = X_train[features_to_encode]
X_train_left = X_train.drop(features_to_encode,axis = 1)

In [213]:
X_train_encoded = pd.get_dummies(X_train_to_be_encoded,drop_first=True)



# now add it to the X_train_left
# but before reset the indices
X_train_left.reset_index(drop = True,inplace = True)
X_train_encoded.reset_index(drop = True,inplace = True)

# concatnating 
X_train_dummies_encoded = pd.concat([X_train_left,X_train_encoded],axis = 1)

In [214]:
print(X_train_dummies_encoded.shape)
X_train_dummies_encoded.head()

(623, 14)


,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Miss,Name_Mr,Name_Mrs,Name_Other,Sex_male,Embarked_Q,Embarked_S
0,115,3,17.000000,0,0,2627,14.4583,1,0,0,0,0,0,0
1,875,2,28.000000,1,0,P/PP 3381,24.0000,0,0,1,0,0,0,0
2,77,3,29.699118,0,0,349208,7.8958,0,1,0,0,1,0,1
3,877,3,20.000000,0,0,7534,9.8458,0,1,0,0,1,0,1
4,675,2,29.699118,0,0,239856,0.0000,0,1,0,0,1,0,1


## We are having 14 columns as before, we are also having columns names, isn't great, we'll found out. Lets repeat the same for test data

In [215]:
X_test_to_be_encoded = X_test[features_to_encode]
X_test_left = X_test.drop(features_to_encode,axis = 1)

X_test_encoded = pd.get_dummies(X_test_to_be_encoded,drop_first=True)



# now add it to the X_train_left
# but before reset the indices
X_test_left.reset_index(drop = True,inplace = True)
X_test_encoded.reset_index(drop = True,inplace = True)

# concatnating 
X_test_dummies_encoded = pd.concat([X_test_left,X_test_encoded],axis = 1)

print(X_test_dummies_encoded.shape)
X_test_dummies_encoded.head()

(243, 13)


,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Miss,Name_Mr,Name_Mrs,Name_Other,Sex_male,Embarked_S
0,863,1,48.000000,0,0,17466,25.9292,0,0,1,0,0,1
1,224,3,29.699118,0,0,349234,7.8958,0,1,0,0,1,1
2,85,2,17.000000,0,0,SO/C 14885,10.5000,1,0,0,0,0,1
3,536,2,7.000000,0,2,F.C.C. 13529,26.2500,1,0,0,0,0,1
4,624,3,21.000000,0,0,350029,7.8542,0,1,0,0,1,1


### We can see that X_Test_dummies_encoded after encoded with dummies has 13 columns, whereas X_train_dummies_encoded has 14 features

## Disadvantage - Not able to learn to fit and transform

In [216]:
# Loosing the features names while using onehotencoder was a loss, but having train and test data with different number of 
# featuer is game over, there is no model which can be trained on 14 features, and then predict values for a data having 13 
# having 13 features

In [218]:
# This happened because we removed initially all the observations from test data having value of embarked == 'Q',
# I did stimulated this, but there's a great chanve that it can be found in real data

# Myencoder

In [219]:
# not we will use my_encoder to see how it can handle the disadvantes of both onehotencoder and dummies

In [222]:
my_encoder = Myencoder()
my_encoder.fit(X_train,features_to_encode)
X_train_encoded_my = my_encoder.transform(X_train)
X_test_encoded_my = my_encoder.transform(X_test)

## And done, all it took was couple of lines of code to cover all the disadvates mentioned above

In [224]:
print(X_train_encoded_my.shape)
X_train_encoded_my.head()

(623, 14)


,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Miss,Name_Mr,Name_Mrs,Name_Other,Sex_male,Embarked_Q,Embarked_S
0,115,3,17.000000,0,0,2627,14.4583,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,875,2,28.000000,1,0,P/PP 3381,24.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,77,3,29.699118,0,0,349208,7.8958,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,877,3,20.000000,0,0,7534,9.8458,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,675,2,29.699118,0,0,239856,0.0000,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [225]:
print(X_test_encoded_my.shape)
X_test_encoded_my.head()

(243, 14)


,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Miss,Name_Mr,Name_Mrs,Name_Other,Sex_male,Embarked_Q,Embarked_S
0,863,1,48.000000,0,0,17466,25.9292,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,224,3,29.699118,0,0,349234,7.8958,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,85,2,17.000000,0,0,SO/C 14885,10.5000,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,536,2,7.000000,0,2,F.C.C. 13529,26.2500,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,624,3,21.000000,0,0,350029,7.8542,0.0,1.0,0.0,0.0,1.0,0.0,1.0


# Hope you enjoyed the notebook :)